# RL coursework, part I (20 pts total)
---

**SN:** 21180859

---

**Due date:** *22nd March, 2022,*

---

Standard UCL policy (including grade deductions) automatically applies for any late submissions.

## How to submit

When you have completed the exercises and everything has finished running, click on 'File' in the menu-bar and then 'Download .ipynb'. This file must be submitted to Moodle named as **`<studentnumber>_RL_part1.ipynb`** before the deadline above, where `<studentnumber>` is your student number.

**Context**

In this assignment, we will take a first look at learning decisions from data.  For this, we will use the multi-armed bandit framework.

**Background reading**

* Sutton and Barto (2018), Chapters 1 to 6
* Lecture slides

**Overview of this assignment**

A) You will use Python to implement several bandit algorithms.

B) You will then run these algorithms on a multi-armed Bernoulli bandit problem, and answer question about their empirical performance.

C) You will then be asked to reason about the behaviour of different algorithms

## Setup

Run each of the cells below, until you reach the next section **Basic Agents**. You do not have to read or understand the code in the **Setup** section.  After running the cells, feel free to fold away the **Setup** section.

In [ ]:
# Import Useful Libraries

import collections
from functools import partial

import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=3, suppress=1)
plt.style.use('seaborn-notebook')

In [ ]:
class BernoulliBandit(object):
  """A stationary multi-armed Bernoulli bandit."""

  def __init__(self, success_probabilities, success_reward=1., fail_reward=0.):
    """Constructor of a stationary Bernoulli bandit.

    Args:
      success_probabilities: A list or numpy array containing the probabilities,
          for each of the arms, of providing a success reward.
      success_reward: The reward on success (default: 1.)
      fail_reward: The reward on failure (default: 0.)
    """
    self._probs = success_probabilities
    self._number_of_arms = len(self._probs)
    self._s = success_reward
    self._f = fail_reward

    ps = np.array(success_probabilities)
    self._values = ps * success_reward + (1 - ps) * fail_reward

  def step(self, action):
    """The step function.

    Args:
      action: An integer or np.int32 that specifies which arm to pull.

    Returns:
      A reward sampled according to the success probability of the selected arm.

    Raises:
      ValueError: when the provided action is out of bounds.
    """
    if action < 0 or action >= self._number_of_arms:
      raise ValueError('Action {} is out of bounds for a '
                       '{}-armed bandit'.format(action, self._number_of_arms))

    success = bool(np.random.random() < self._probs[action])
    reward = success * self._s + (not success) * self._f
    return reward

  def regret(self, action):
    """Computes the regret for the given action."""
    return self._values.max() - self._values[action]

  def optimal_value(self):
    """Computes the regret for the given action."""
    return self._values.max()


In [ ]:
class NonStationaryBandit(object):
  """A non-stationary multi-armed Bernoulli bandit."""

  def __init__(self, success_probabilities,
               success_reward=1., fail_reward=0., change_point=800,
               change_is_good=True):
    """Constructor of a non-stationary Bernoulli bandit.

    Args:
      success_probabilities: A list or numpy array containing the probabilities,
          for each of the arms, of providing a success reward.
      success_reward: The reward on success (default: 1.)
      fail_reward: The reward on failure (default: 0.)
      change_point: The number of steps before the rewards change.
      change_is_good: Whether the rewards go up (if True), or flip (if False).
    """
    self._probs = success_probabilities
    self._number_of_arms = len(self._probs)
    self._s = success_reward
    self._f = fail_reward
    self._change_point = change_point
    self._change_is_good = change_is_good
    self._number_of_steps_so_far = 0

    ps = np.array(success_probabilities)
    self._values = ps * success_reward + (1 - ps) * fail_reward

  def step(self, action):
    """The step function.

    Args:
      action: An integer or np.int32 that specifies which arm to pull.

    Returns:
      A reward sampled according to the success probability of the selected arm.

    Raises:
      ValueError: when the provided action is out of bounds.
    """
    if action < 0 or action >= self._number_of_arms:
      raise ValueError('Action {} is out of bounds for a '
                       '{}-armed bandit'.format(action, self._number_of_arms))

    self._number_of_steps_so_far += 1
    success = bool(np.random.random() < self._probs[action])
    reward = success * self._s + (not success) * self._f
    
    if self._number_of_steps_so_far == self._change_point:
      # After some number of steps, the rewards are inverted
      #
      #  ``The past was alterable. The past never had been altered. Oceania was
      #    at war with Eastasia. Oceania had always been at war with Eastasia.``
      #            - 1984, Orwell (1949).
      reward_dif = (self._s - self._f)
      if self._change_is_good:
        self._f = self._s + reward_dif
      else:
        self._s -= reward_dif
        self._f += reward_dif
      
      # Recompute expected values when the rewards change
      ps = np.array(self._probs)
      self._values = ps * self._s + (1 - ps) * self._f

    return reward
  
  def regret(self, action):
    """Computes the regret for the given action."""
    return self._values.max() - self._values[action]
  
  def optimal_value(self):
    """Computes the regret for the given action."""
    return self._values.max()

In [ ]:
# Helper functions

def smooth(array, smoothing_horizon=100., initial_value=0.):
  """Smoothing function for plotting."""
  smoothed_array = []
  value = initial_value
  b = 1./smoothing_horizon
  m = 1.
  for x in array:
    m *= 1. - b
    lr = b/(1 - m)
    value += lr*(x - value)
    smoothed_array.append(value)
  return np.array(smoothed_array)

def plot(algs, plot_data, repetitions=30):
  """Plot results of a bandit experiment."""
  algs_per_row = 4
  n_algs = len(algs)
  n_rows = (n_algs - 2)//algs_per_row + 1
  fig = plt.figure(figsize=(10, 4*n_rows))
  fig.subplots_adjust(wspace=0.3, hspace=0.35)
  clrs = ['#000000', '#00bb88', '#0033ff', '#aa3399', '#ff6600']
  lss = ['--', '-', '-', '-', '-']
  for i, p in enumerate(plot_data):
    for c in range(n_rows):
      ax = fig.add_subplot(n_rows, len(plot_data), i + 1 + c*len(plot_data))
      ax.grid(0)

      current_algs = [algs[0]] + algs[c*algs_per_row + 1:(c + 1)*algs_per_row + 1]
      for alg, clr, ls in zip(current_algs, clrs, lss):
        data = p.data[alg.name]
        m = smooth(np.mean(data, axis=0))
        s = np.std(smooth(data.T).T, axis=0)/np.sqrt(repetitions)
        if p.log_plot:
          line = plt.semilogy(m, alpha=0.7, label=alg.name,
                              color=clr, ls=ls, lw=3)[0]
        else:
          line = plt.plot(m, alpha=0.7, label=alg.name,
                          color=clr, ls=ls, lw=3)[0]
          plt.fill_between(range(len(m)), m + s, m - s,
                           color=line.get_color(), alpha=0.2)
      if p.opt_values is not None:
        plt.plot(p.opt_values[current_algs[0].name][0], ':', alpha=0.5,
                 label='optimal')

      ax.set_facecolor('white')
      ax.tick_params(axis="both", which="both", bottom="off", top="off",
                     labelbottom="on", left="off", right="off", labelleft="on")
      ax.spines["top"].set_visible(False)
      ax.spines["bottom"].set(visible=True, color='black', lw=1)
      ax.spines["right"].set_visible(False)
      ax.spines["left"].set(visible=True, color='black', lw=1)
      ax.get_xaxis().tick_bottom()
      ax.get_yaxis().tick_left()

      data = np.array([smooth(np.mean(d, axis=0)) for d in p.data.values()])
      
      if p.log_plot:
        start, end = calculate_lims(data, p.log_plot)
        start = np.floor(np.log10(start))
        end = np.ceil(np.log10(end))
        ticks = [_*10**__
                 for _ in [1., 2., 3., 5.]
                 for __ in [-2., -1., 0.]]
        labels = [r'${:1.2f}$'.format(_*10** __)
                  for _ in [1, 2, 3, 5]
                  for __ in [-2, -1, 0]]
        plt.yticks(ticks, labels)
      plt.ylim(calculate_lims(data, p.log_plot))
      plt.locator_params(axis='x', nbins=4)
      
      plt.title(p.title)
      if i == len(plot_data) - 1:
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

def run_experiment(bandit_constructor, algs, repetitions, number_of_steps):
  """Run multiple repetitions of a bandit experiment."""
  reward_dict = {}
  regret_dict = {}
  optimal_value_dict = {}

  for alg in algs:
    reward_dict[alg.name] = np.zeros((repetitions, number_of_steps))
    regret_dict[alg.name] = np.zeros((repetitions, number_of_steps))
    optimal_value_dict[alg.name] = np.zeros((repetitions, number_of_steps))

    for _rep in range(repetitions):
      bandit = bandit_constructor()
      alg.reset()

      action = None
      reward = None
      for _step in range(number_of_steps):
        action = alg.step(action, reward)
        reward = bandit.step(action)
        regret = bandit.regret(action)
        optimal_value = bandit.optimal_value()

        reward_dict[alg.name][_rep, _step] = reward
        regret_dict[alg.name][_rep, _step] = regret
        optimal_value_dict[alg.name][_rep, _step] = optimal_value

  return reward_dict, regret_dict, optimal_value_dict


def train_agents(agents, number_of_arms, number_of_steps, repetitions=100,
                 success_reward=1., fail_reward=0.,
                 bandit_class=BernoulliBandit):

  success_probabilities = np.arange(0.3, 0.7 + 1e-6, 0.4/(number_of_arms - 1))

  bandit_constructor = partial(bandit_class,
                               success_probabilities=success_probabilities,
                               success_reward=success_reward,
                               fail_reward=fail_reward)
  rewards, regrets, opt_values = run_experiment(
      bandit_constructor, agents, repetitions, number_of_steps)

  smoothed_rewards = {}
  for agent, rs in rewards.items():
    smoothed_rewards[agent] = np.array(rs)

  PlotData = collections.namedtuple('PlotData',
                                    ['title', 'data', 'opt_values', 'log_plot'])
  total_regrets = dict([(k, np.cumsum(v, axis=1)) for k, v in regrets.items()])
  plot_data = [
      PlotData(title='Smoothed rewards', data=smoothed_rewards,
               opt_values=opt_values, log_plot=False),
      PlotData(title='Current Regret', data=regrets, opt_values=None,
               log_plot=True),
      PlotData(title='Total Regret', data=total_regrets, opt_values=None,
               log_plot=False),
  ]

  plot(agents, plot_data, repetitions)

def calculate_lims(data, log_plot=False):
  y_min = np.min(data)
  y_max = np.max(data)
  diff = y_max - y_min
  if log_plot:
    y_min = 0.9*y_min
    y_max = 1.1*y_max
  else:
    y_min = y_min - 0.05*diff
    y_max = y_max + 0.05*diff
  return y_min, y_max

def argmax(array):
  """Returns the maximal element, breaking ties randomly."""
  return np.random.choice(np.flatnonzero(array == array.max()))

# A) Agent implementations


All agents should be in pure Python/NumPy.

You cannot use any AutoDiff packages (Jax, TF, PyTorch, etc.)

Each agent, should implement the following methods:

**`step(self, previous_action, reward)`:**

Should update the statistics by updating the value for the previous_action towards the observed reward.

(Note: make sure this can handle the case that previous_action=None, in which case no statistics should be updated.)

(Hint: you can split this into two steps: 1. update values, 2. get new action.  Make sure you update the values before selecting a new action.)

**`reset(self)`:**

Resets statistics (should be equivalent to constructing a new agent from scratch).

Make sure that the initial values (after a reset) are all zero.

**`__init__(self, name, number_of_arms, *args)`:**

The `__init__` should take at least an argument `number_of_arms`, and (potentially) agent specific args.

## Example agent

The following code block contains an example random agent.

In [ ]:
class Random(object):
  """A random agent.

  This agent returns an action between 0 and 'number_of_arms', uniformly at
  random. The 'previous_action' argument of 'step' is ignored.
  """

  def __init__(self, name, number_of_arms):
    """Initialise the agent.
    
    Sets the name to `random`, and stores the number of arms. (In multi-armed
    bandits `arm` is just another word for `action`.)
    """
    self._number_of_arms = number_of_arms
    self.name = name

  def step(self, unused_previous_action, unused_reward):
    """Returns a random action.
    
    The inputs are ignored, but this function still requires an action and a
    reward, to have the same interface as other agents who may use these inputs
    to learn.
    """
    return np.random.randint(self._number_of_arms)

  def reset(self):
    pass


## Q1 [2 pts]
Implement a UCB agent.

The `bonus_multiplier` is the parameter $c$ from the slides.

In [ ]:
class UCB(object):
  def __init__(self, name, number_of_arms, bonus_multiplier, t = 0, Na = [], Qa = []):
    
    self.number_of_arms = number_of_arms
    self._bonus_multiplier = bonus_multiplier
    self.name = name
    self.t = 0 # step is needed as it's used in choosing what action to take
    self.Na = np.zeros(self.number_of_arms) #  this array holds the information of how often each action has been taken
    self.Qa = np.zeros(self.number_of_arms) # this array holds the value estimates for each action

    #self.reset()
    
  def step(self, previous_action, reward):
  # [my code] 
    UCBTerm = np.zeros(self.number_of_arms)
    if previous_action == None and reward == None:
      action = 0
      self.t += 1
      self.Na[action] += 1    
      return action
    else:
      self.Qa[previous_action] += (1/self.Na[previous_action])*(reward-self.Qa[previous_action])
      if self.t <= self.number_of_arms and self.t < self.number_of_arms-1:
        action = previous_action + 1
      elif self.t == self.number_of_arms-1:
        action = self.t
      else:
        for a in range(self.number_of_arms):
          UCBTerm[a] = UCBTerm[a] = self.Qa[a] + self._bonus_multiplier*np.sqrt(np.log(self.t)/self.Na[a]) 
        action = np.argmax(UCBTerm)
      self.t += 1
      self.Na[action] += 1
      return action
    
  def reset(self):
    self.number_of_arms = self.number_of_arms
    self._bonus_multiplier = self._bonus_multiplier
    self.name = self.name
    self.t = 0 
    self.Na = np.zeros(self.number_of_arms)  
    self.Qa = np.zeros(self.number_of_arms)

## Q2 [1 pt]
Implement an $\epsilon$-greedy agent.

This agent should be able to support time-changing $\epsilon$ schedules.

Thus, your agent should accept both constants and callables as constructor argument `epsilon`; callables are used to decay the $\epsilon$ parameter over time, for instance according to a polynomial schedule: $\epsilon_t = t^{-\eta}$ with $\eta \in [0, 1]$).


If multiple actions have the same value, ties should be broken randomly.

In [ ]:
class EpsilonGreedy(object):
  """An epsilon-greedy agent.

  This agent returns an action between 0 and 'number_of_arms'; with probability
  `(1-epsilon)` it chooses the action with the highest estimated value, while
  with probability `epsilon` it samples an action uniformly at random.
  """

  def __init__(self, name, number_of_arms, epsilon=0.1, Qa = [], Na = [], t= 0):
    self.name = name
    self.number_of_arms = number_of_arms
    self.epsilon = epsilon
    self.Qa = np.zeros(self.number_of_arms)
    self.Na = np.zeros(self.number_of_arms)
    self.t = t
    #self.reset()

  def step(self, previous_action, reward):
    """Update the learnt statistics and return an action.

    A single call to step uses the provided reward to update the value of the
    taken action (which is also provided as an input), and returns an action.
    The action is either uniformly random (with probability epsilon), or greedy
    (with probability 1 - epsilon).

    If the input action is None (typically on the first call to step), then no
    statistics are updated, but an action is still returned.
    """
    if previous_action != None:
      self.Na[previous_action] += 1
      self.Qa[previous_action] += (1. / self.Na[previous_action]) * (reward - self.Qa[previous_action])
    self.t += 1

    epsilon = self.epsilon(self.t) if callable(self.epsilon) else self.epsilon
    if np.random.rand() < epsilon:
      action = np.random.randint(self.number_of_arms)
    else:
      action = argmax(self.Qa)
    return action   
   

  def reset(self):
    self.name = self.name
    self.number_of_arms = self.number_of_arms
    self.epsilon = self.epsilon
    self.Qa = np.zeros(self.number_of_arms)
    self.Na = np.zeros(self.number_of_arms)
    self.t = 0


## Q3 [2 pts]
Implement a REINFORCE agent.

While `softmax` distributions are a common parametrization for policies over discrete action-spaces, they are not the only choice. In this exercise we ask you to implement REINFORCE with the `square-max` policy parameterization. With this parametrisation the probabilities depend on the action preferences $p(\cdot)$ according to the expression:

$$\pi(a) = \frac{p(a)^2}{\sum_b p(b)^2}\,.$$

Implement a REINFORCE policy-gradient method for updating the preferences under this policy distribution. The action preferences are stored separately, so that for each action $a$ the preference $p(a)$ is a single value that you directly update.

The agent should be able to use a baseline or not (as defined in the constructor). The `step_size` parameter $\alpha$ used to update the policy must also be configurable in the constructor.

The baseline should track the average reward so far, using the same `step_size` used to update the policy.

The `step_size` and whether or not a baseline is used are defined in the constructor by feeding additional arguments in place of `...` below.


In [ ]:
class REINFORCE(object):
  def __init__(self, name, number_of_arms, step_size=0.1, baseline=False, preferences = [], t = 0, avg_reward = 0):
    self.name = name
    self.number_of_arms = number_of_arms
    self.preferences = np.ones(number_of_arms)
    self.baseline = baseline
    self.t = t
    self.avg_reward = avg_reward
    self.step_size = step_size
    #self.reset()

  def step(self, previous_action, reward):
    
    if previous_action == None:
        probabilities = self.preferences**2/(sum(self.preferences**2))
        action = np.random.choice(np.arange(0, self.number_of_arms), p=probabilities)
        self.t += 1
        return action 
    else:
        probabilities = self.preferences**2/(sum(self.preferences**2))
        if self.baseline:
            if self.step == 1:
              self.avg_reward = reward
            else:
              self.avg_reward = (1/(self.t+1))*(reward - self.avg_reward)
            for a in range(self.number_of_arms):
              if a == previous_action:
                self.preferences[a] += self.step_size*(reward - self.avg_reward)*(1-probabilities[a])
              else:
                self.preferences[a] -= self.step_size*(reward - self.avg_reward)*probabilities[a]
            probabilities = self.preferences**2/(sum(self.preferences**2))
            action = np.random.choice(np.arange(0, self.number_of_arms), p=probabilities)
            self.t += 1
            return action 
        else:
            for a in range(self.number_of_arms):
                  if a == previous_action:
                    self.preferences[a] += self.step_size*(reward )*(1-probabilities[a])
                  else:
                    self.preferences[a] -= self.step_size*(reward )*probabilities[a]
            probabilities = self.preferences**2/(sum(self.preferences**2))
            action = np.random.choice(np.arange(0, self.number_of_arms), p=probabilities)
            self.t += 1
            return action 

  def reset(self):
    self.name = self.name
    self.number_of_arms = self.number_of_arms
    self.preferences = np.ones(self.number_of_arms)
    self.baseline = self.baseline
    self.t = 0
    self.step_size=self.step_size
    self.avg_reward = 0




# B) Experiments

**Run the cell below to train the agents and generate the plots for the first experiment.**

Trains the agents on a Bernoulli bandit problem with 5 arms,
with a reward on success of 1, and a reward on failure of 0.

## Experiment 1: Bernoulli bandit

In [ ]:
%%capture experiment1

number_of_arms = 5
number_of_steps = 1000

agents = [
    Random(
        "random",
        number_of_arms),
    EpsilonGreedy(
        r"$\epsilon$-greedy with $\epsilon=0$",
        number_of_arms,
        epsilon=0.),
    EpsilonGreedy(
        r"$\epsilon$-greedy with $\epsilon=0.1$",
        number_of_arms,
        epsilon=0.1),

    EpsilonGreedy(
        r"$\epsilon$-greedy with $\epsilon_t=1/t$",
        number_of_arms,
        epsilon=lambda t: 1./t),
    EpsilonGreedy(
        r"$\epsilon$-greedy with $\epsilon_t=1/\sqrt{t}$",
        number_of_arms,
        epsilon=lambda t: 1./t**0.5),

    UCB("UCB",
        number_of_arms,
        bonus_multiplier=1/np.sqrt(2)),
    REINFORCE(
        r"REINFORCE, $\alpha=0.1$",
        number_of_arms,
        step_size=0.1,
        baseline=False),
    REINFORCE(
        r"REINFORCE with baseline, $\alpha=0.1$",
        number_of_arms,
        step_size=0.1,
        baseline=True),
]

train_agents(agents, number_of_arms, number_of_steps)

In [ ]:
experiment1.show()

## Q4 [4 pts total]
(Answer inline in the markdown below each question, **within this text cell**.)

**[2 pts]**
For each algorithm in the plots above, explain whether or not we should be expected it to be good in general, in terms of total regret.

- Random algorithm: We expected for the random algorithm to perform very bad since it doesn't learn from the data like non-random (learning) algorithms and hence doesn't improve its choices and hence why its total regret is expected to be in the set of the worser algorithms.
- $\epsilon$ - greedy with $\epsilon = 0$ algorithm: This algorithm doesn't pick any actions which don't yield the highest expected reward and hence it may be picking sub-optimal actions often, yielding bad results.
- $\epsilon$ - greedy with $\epsilon = 0.1$ algorithm: This algorithm picks a random action 10% of the time, hence exploring other actions than the action with the highest expected reward with probability $\frac{\text{number of actions -1}}{\text{number of actions}}$ (since it may pick the action with the highest expected reward as well). This algorithm is expected to perform well although it will have slow convergence.
- $\epsilon$ - greedy with $\epsilon=\frac{1}{t}$ and $\epsilon$ - greedy with $\epsilon=\frac{1}{\sqrt{t}}$ algorithms: These algorithms explore less as time passess. These algorithms are expected to perform well since for the first steps the algorithms get to explore the actions often, and once they explore all of the actions they gets a better estimate of the expected rewards and stick to those actions.
- UCB algorithm: This algorithm was expected to perform well, because the algorithm explores efficiently i.e., it prioritizes actions which haven't been explored (even if those actions gave bad rewards) but it doesn't prioritize actions which gave low rewards consistently. This prioritization ensures that all of the actions are explored well enough so that the regret is minimal.
- Gradient Algorithms with and without baseline: These algorithms were expected to perform well because they pick actions according to preference which are updated via gradient ascent. Since they don't act greedily, but choose their actions according to a probability distribution, it ensures that sufficient exploration happens and hence these algorithms are expected to perform well.


**[2 pts]** Explain the relative ranking of the $\epsilon$-greedy algorithms in this experiment.

- $\epsilon = 0$ vs $\epsilon = \frac{1}{t}$


 The worst performing algorithm is the one with $\epsilon =0$ since the algorithm chooses the greedy action 100% of the time. This makes it hard for the agent to know the estimates of the other actions more accurately (because it doesn't pick those actions whilst acting greedily). If we compare it to the algorithms with $\epsilon=\frac{1}{t}$, it looks like as if these algorithms should behave (in terms of greediness) very similarly the more time increases. But the fact that $\epsilon=\frac{1}{t}$ explored aggressively during the first steps ensured that it had more information regarding the estimates of the other actions, and hence the better performance than $\epsilon=0$.

- $\epsilon = \frac{1}{t}$ vs $\epsilon = 0.1$

$\epsilon = 0.1$ performed better than $\epsilon = \frac{1}{t}$ since $\epsilon=\frac{1}{t}$ became greedy (exploration probability $<0.1$ after the 10th step) very quickly and apparently with 5 arms, it is not sufficient to ensure enough exploration to beat $\epsilon=0.1$'s performance. If more steps were given, $\epsilon=\frac{1}{t}$ would eventually give better results, as it would choose the greedy action (and maybe optimal) more often.

- $\epsilon = \frac{1}{t}$ vs $\epsilon = \frac{1}{\sqrt{t}}$

The best performing algorithm, $\epsilon = \frac{1}{\sqrt{t}}$ performed the best because it had more exploration during the initial steps, and hence it got better estimates early on. These esimates were better than the $\epsilon=\frac{1}{t}$'s algorithm because instead of exploring 10 steps before having its exploration probability cut down to $0.1$, it explored $100$ steps. And even after that it explored more and hence increased its confidence in its estimates. Eventually though, $\epsilon=\frac{1}{t}$ would give better results (on average) because it would choose the greedy action more often.

- $\epsilon = \frac{1}{\sqrt{t}} $ vs $\epsilon = 0.1$

These algorithms performed the best, however $\epsilon = \frac{1}{\sqrt{t}} $  did better because it explored more during the inital steps and decreased its exploration once the estimates became more concrete. $\epsilon=0.1$ was still exploring (with probability $0.1$) after getting good estimates of all the actions and this exploration became detrimental to its total regret.

## Experiment 2: reward = 0 on success, reward = -1 on failure.

**Run the cell below to train the agents and generate the plots for the second experiment.**
Reruns experiment 1 but on a different bernoulli bandit problem with 5 arms,
with a reward on success of 0, and a reward on failure of -1.

---



In [ ]:
%%capture experiment2
number_of_arms = 5
number_of_steps = 1000

train_agents(agents, number_of_arms, number_of_steps,
             success_reward=0., fail_reward=-1.)

In [ ]:
experiment2.show()

## Q5 [2 pts]
For each algorithm, note whether the performance changed significantly compared to the **experiment 1**, and explain why it did or did not.

(Use at most two sentences per algorithm).

 - $\epsilon = 0$: This algorithm's performance changed significantly because before when it was getting $0$ reward for staying at a sub-optimal action, the estimates didn't change much for this action relative to the others actions, but now that it is getting punished for staying at a sub-optimal action (and hence the estimate relative to the other actions decreases), it changes its actions faster, hence increasing the chance that it will pick the optimal action quicker.

 - $\epsilon = 0.1$ The algorithm's performance has changed, but not significantly, because before it was exploring different actions (and getting good estimates of the non-greedy actions) but now that it is getting punished it changed its course of action quicker, yielding better results. 

 - $\epsilon = \frac{1}{t}$ - The algorithm's performance increased slightly for the same reason as above.

 - UCB algorithm: The algorithm's performance increased slightly for the same reason as above.


 - Gradient algorithms: These algorithms' performances didn't change or increased very slightly. This is because before (with reward-failure = 1,0) it was choosing its actions by comparing an action to all other actions and now it it doing the same, meaning that the rewards themselves don't play an important role in choosing an action as opposed to the algorithms which base their choice on the average of the rewards (for a specific action) so far.

 


## Run the following cells

## Experiment 3: Non-stationary bandit
 * Reward on `failure` changes from 0 to +2.
 * Reward on `success` remains at +1.


In [ ]:
%%capture experiment3

number_of_arms = 3
number_of_steps = 1984
agents = [
    Random(
        "random",
        number_of_arms),
    EpsilonGreedy(
        r"$\epsilon$-greedy with $\epsilon=0.1$",
        number_of_arms,
        epsilon=0.1),
    EpsilonGreedy(
        r"$\epsilon$-greedy with $\epsilon_t=1/\sqrt{t}$",
        number_of_arms,
        epsilon=lambda t: 1./t**0.5),
    UCB("UCB",
        number_of_arms,
        bonus_multiplier=1/np.sqrt(2)),
    REINFORCE(
        r"REINFORCE with baseline, $\alpha=0.1$",
        number_of_arms,
        step_size=0.1,
        baseline=True),

]

roving_bandit_class = partial(NonStationaryBandit, change_is_good=True)
train_agents(agents, number_of_arms, number_of_steps,
             bandit_class=roving_bandit_class)

## Experiment 4: Non-stationary bandit
 * Reward on `failure` changes from 0 to +1.
 * Reward on `success` changes from +1 to 0.


In [ ]:
%%capture experiment4

number_of_arms = 3
number_of_steps = 1984


roving_bandit_class = partial(NonStationaryBandit, change_is_good=False)
train_agents(agents, number_of_arms, number_of_steps,
             bandit_class=roving_bandit_class)

In [ ]:
experiment3.show()

**[2 pts]** In **experiment 3** explain the ranking in current regret after the change in rewards for all algorithms.

> We can see that UCB starts performing the worst, followed by $\epsilon = \frac{1}{\sqrt{t}}$, the gradient algorithm and finally the best performing algorithm becomes $\epsilon = 0.1$. The reason UCB starts performing the worst is because it doesn't stop choosing the actions which were optimal until step 800 and since the reward for success doesn't change, the estimates for these actions don't decrease relative to the other actions, and hence these actions (which were good until step 800) keep on being picked. On the other hand we see that $\epsilon = 0.1$ performs the best, because it explores more often than the other algorithms and adapts quickly. The gradient algorithm performs better than the others because there is a higher probability to choose a non-greedy action and hence for the agent to update its preferences quicker. Finally the $\epsilon = \frac{1}{\sqrt{t}}$ algorithm has a similar (but not the same) problem with that of the UCB algorithm, in that it doesn't explore other actions because the exploration probability becomes very small due to $t$ being larger than $800$ and hence it continues picking the sub-optimal actions.

In [ ]:
experiment4.show()

## Q6 [9 pts total]

Observe the reward and regret curves above.  After 800 steps, the rewards change. In **experiment 3** `success` continues to yield a reward of +1, but `failure` changes from a reward of 0 to a reward of +2.  In **experiment 4**, `success` is now worth 0 and `failure` is worth +1.

Below, we ask for explanations.  Answer each question briefly, using at most three sentences per question.

**[2 pts]** In **experiment 3** explain the ranking in current regret after the change in rewards for all algorithms.

> We can see that UCB starts performing the worst, followed by $\epsilon = \frac{1}{\sqrt{t}}$, the gradient algorithm and finally the best performing algorithm becomes $\epsilon = 0.1$. The reason UCB starts performing the worst is because it doesn't stop choosing the actions which were optimal until step 800 and since the reward for success doesn't change, the estimates for these actions don't decrease relative to the other actions, and hence these actions (which were good until step 800) keep on being picked. On the other hand we see that $\epsilon = 0.1$ performs the best, because it explores more often than the other algorithms and adapts quickly. The gradient algorithm performs better than the others because there is a higher probability to choose a non-greedy action and hence for the agent to update its preferences quicker. Finally the $\epsilon = \frac{1}{\sqrt{t}}$ algorithm has a similar (but not the same) problem with that of the UCB algorithm, in that it doesn't explore other actions because the exploration probability becomes very small due to $t$ being larger than $800$ and hence it continues picking the sub-optimal actions.

**[2 pts]** In **experiment 4** explain the ranking in current regret after the change in rewards for all algorithms.

> We can see that the ranking is the same (as in experiment 3) except that the UCB algorithm is now the best performing one, instead of the worst. The reason the other 3 have the same relative ranking is because the failure to success ratio didn't change. But the relative change of success  in this experiment (the decrease from 1 to 0) impacted the UCB algorithm significantly, because now since it was choosing those actions which were sub-optimal (after the 800'th step) and since the estimates decreased (due to the success reward decreasing), it considered other actions (because their estimates didn't decrease since they were not being picked) and quickly adapted, causing it to be the best performing algorithm, as observed in experiments 1 and 2.

**[2 pts]** Explain how and why the current-regret curve for UCB in **experiment 3** differs from the curve in **experiment 4**.

> Explained in the previous answers.

**[3 pts]** In general, if rewards can be non-stationary, and we don't know the exact nature of the non-stationarity, how could we modify UCB to perform better?   Be specific and concise.

> There are two algorithms that have been discussed which could help UCB be adapted to the non-staitonary case: the dicounted and sliding-window UCB. These algorithms explore more as time passes, compared to the original UCB. They do this by giving less importance to past rewards. The discounted UCB uses a discount factor $\gamma\in (0,1)$ to track reward estimates and this way it gives more weights to recent rewards in it estimates. It also gives more confidence to the most recent rewards and replaces the action count in its uncertainty term with a count that discounts past actions exponentially with $\gamma$. Sliding-window UCB considers the last actions and doesn't consider all of them (and this is the difference, these algorithms are very similar).
